In [1]:
import ee, eemont, geemap
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
import altair as alt
import numpy as np
import folium
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline

import geemap.eefolium as emap

In [2]:
#Use GEE to extract NDVI data for croplands in Kenya
#Get boundary for Kenya
#Clip global NDVI data to Kenya
#Mask Kenya NDVI data by cropland layer
#Extract time-series of NDVI for cropland pixels in Kenya
#

import ee
ee.Authenticate()
ee.Initialize()


Enter verification code: 4/1AX4XfWhufx3wCLWH_8WqECMYWCfwkNvcUnoDzj8sx62udwhj-xXCVtyl8_g

Successfully saved authorization token.


In [5]:
#Folium library 

import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [21]:
#get Kenya boundaries by calling countires api
from IPython.display import Image


countries = ee.FeatureCollection('FAO/GAUL/2015/level1').select(['ADM0_NAME', 'ADM1_NAME'])

# Filter the feature collection to subset Kenya Boundaries.

Kenya= countries.filter(ee.Filter.eq('ADM0_NAME', 'Kenya'))



Central = Kenya.filter(ee.Filter.eq('ADM1_NAME', 'Central'))


#Central = Kenya.filter(ee.Filter.eq('ADM1_NAME', 'Nairobi'))


import geemap.eefolium as emap

def kenya_map():
    
    Map = emap.Map()
    Map.addLayer(Central)
    Map.addLayerControl()
    Map.setCenter(37.9083, 0.1769,6)
    return Map

kenya_map()



In [22]:
im_coll = ee.ImageCollection('MODIS/MOD09GA_006_NDVI').select('NDVI').filter(ee.Filter.date('2018-04-01', '2018-05-01')) ;
def mask_map():
    
    Map = emap.Map()
    Map.addLayer(im_coll)
    Map.addLayerControl()
    Map.setCenter(37.9083, 0.1769,6)
    return Map

mask_map()






In [23]:
#clips the image then map the image to the imagecollection 
def clip_img(img):
    """Clips a list of images."""
    return ee.Image(img).clip(Central)


clipped_im = im_coll.map(clip_img)


def clip_map():
    
    Map = emap.Map()
    Map.addLayer(clipped_im)
    Map.addLayerControl()
    Map.setCenter(37.9083, 0.1769,6)
    return Map

clip_map()




In [24]:
im_crop = ee.Image('USGS/GFSAD1000_V1').select("landcover")

def crop_map():
    
    Map = emap.Map()
    Map.addLayer(im_crop)
    Map.addLayerControl()
    Map.setCenter(37.9083, 0.1769,6)
    return Map

crop_map()


In [28]:
im_masked=clipped_im.map(lambda x: x.updateMask(im_crop).neq(0))


def mask_water_layer_map():
    
    Map = emap.Map()
    #Map.addLayer(Central)
    Map.addLayer(im_masked)
    Map.addLayerControl()
    Map.setCenter(37.9083, 0.1769,6)
    return Map

mask_water_layer_map()

In [29]:
# perform region reduction 

def create_reduce_region_function(geometry,
                                  reducer=ee.Reducer.median(),
                                  scale=500,
                                  crs='EPSG:4326',
                                  bestEffort=True,
                                  maxPixels=1e13,
                                  tileScale=4):
    
    def reduce_region_function(img):
        """Applies the ee.Image.reduceRegion() method.

    Args:
      img:
        An ee.Image to reduce to a statistic by region.

    Returns:
      An ee.Feature that contains properties representing the image region
      reduction results per band and the image timestamp formatted as
      milliseconds from Unix epoch (included to enable time series plotting).
    """
        stat = img.reduceRegion(
            reducer=reducer,
            geometry=geometry,
            scale=scale,
            crs=crs,
            bestEffort=bestEffort,
            maxPixels=maxPixels,
            tileScale=tileScale)
    

        return ee.Feature(geometry, stat).set({'system:time_start': ee.Date('2018-04-01')})
    return reduce_region_function

In [30]:
#reduce data 

#var NDVI_may_2014 = NDVImap_may_2014.reduce(ee.Reducer.median()).set({'system:time_start': may_date});

reduce_masked = create_reduce_region_function(
    geometry=Central, reducer=ee.Reducer.mean(), scale=1000, crs='EPSG:4326')

kenya_stat_fc = ee.FeatureCollection(im_masked.map(reduce_masked)).filter(
    ee.Filter.notNull(im_masked.first().bandNames()))#.filterDate(ee.Date('2019-09-01'),ee.Date('2020-01-31'))

import geemap.eefolium as emap
Map = emap.Map()
Map.addLayer(kenya_stat_fc)


Map.addLayerControl()

Map


In [51]:
#reduceRegion doesn't return any output if the image doesn't intersect
#with the point or if the image is masked out due to cloud
#If there was no ndvi value found, we set the ndvi to a NoData value -9999

ndvi = ee.List([kenya_stat_fc.get('ndvi'), -9999]).reduce(ee.Reducer.firstNonNull())
 
ndvi.getInfo()

-9999

In [53]:
Map = geemap.Map()



vis_params = {
  'min': 0.0,
  'max': 1.0,
  'palette': [
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
  ],
}

first_image = clipped_im.first()

Map.addLayer(first_image, vis_params, "First image")
Map.setCenter(-7.03125, 31.0529339857, 2)
Map

Map(center=[31.0529339857, -7.03125], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=H…

In [34]:
modis = im_coll.toBands()
Map.addLayer(modis, {}, "MODIS Time series", False)

In [35]:
months = ee.List.sequence(1,12)
years = ee.List.sequence(2016, 2019)
AOI =  ee.Geometry.Polygon(
[[[-56.37397887990624, -12.737207954893526],
[-56.37397887990624, -13.300834158918619],
[-55.37113311574608, -13.300834158918619],
[-55.37113311574608, -12.737207954893526]]])

In [36]:
im_coll = ee.ImageCollection('MODIS/MOD09GA_006_NDVI').select('NDVI').filterDate('2018-04-01', '2018-05-01').filterBounds(Central)

modis_ndvi = im_coll.median().clip(Central)
mean_ndvi = im_coll.mean().clip(Central)

In [42]:
import folium
from folium import plugins
from IPython.display import Image
import geopandas as gpd
import json
print(folium.__version__)
from ipygee import*
import math
import pandas as pd
from tslearn.clustering import TimeSeriesKMeans
from tslearn.utils import to_time_series_dataset

0.12.1.post1


/Users/beaconeconomics/opt/anaconda3/lib/python3.7/site-packages/tslearn/clustering/kmeans.py:17: UserWarning: Scikit-learn <0.24 will be deprecated in a future release of tslearn
  "Scikit-learn <0.24 will be deprecated in a "


In [43]:
def monthly(collection):
    img_coll = ee.ImageCollection([])
    for y in years.getInfo():
        for m in months.getInfo():
            filtered = collection.filter(ee.Filter.calendarRange(y,y,'year')).filter(ee.Filter.calendarRange(m, m, 'month'))
            filtered = filtered.median()
            img_coll = img_coll.merge(filtered.set('year', y)
                             .set('month', m)
                             .set('system:time_start',
                             ee.Date.fromYMD(y, m, 1)
                             .getInfo()['value']))
    return img_coll
Monthly_MD = monthly(im_coll)



In [45]:
Point_1 = ee.Geometry.Point([-0.7635,36.8207])
MD_ndvi = chart.Image.series(**{'imageCollection': Monthly_MD,
'region': Point_1,
'reducer': ee.Reducer.max(),
'scale': 500,
'xProperty': 'system:time_start'})
MD_ndvi.renderWidget(width='50%')

EEException: User memory limit exceeded.